# Bilateral case comparison for CCVA

## Resume

In that notebook, we will implement the computation of the first-order approximation of the defined TVA written in the paper of Crépey and Song [Counterparty Risk Modeling: Beyond Immersion
](http://grozny.maths.univ-evry.fr/pages_perso/crepey/papers/marks-SUBMITTED.pdf)

Remind the first-order approximation formula of the reduced form:

\begin{align*}
\Theta_0^{(1)} =& \, \mathbb{E} \left[ 
\int_{0}^{\bar{T}} \alpha_s \tilde{f}_s(0) ds
\right] \\
=& \, \mathbb{E} \left( 1_{\tau \leq \bar{T}} \frac{1}{\lambda e^{-\lambda \tau}} \alpha_{\tau} \tilde{f}_{\tau}(0) \right)
\end{align*}

with the $\tilde{f}_{.}(0)$ function is given by:

\begin{align*}
\tilde{f}_t(0) = &(1-R_c) \times \gamma^c \times (P_t + \Delta_t - \Gamma_t^c)^+ \\
- &(1-R_b) \times \gamma^b \times (P_t + \Delta_t - \Gamma_t^b)^- \\
+ &(c_t \Gamma_t^f + \tilde{\lambda}_t (P_t - \Gamma_t^f)^+
- \lambda_t (P_t - \Gamma_t^f)^-) \\
+ &\, k_t K_t
\end{align*}

with:  

- $\beta_t = \exp ( - \int_0^t r_s ds )$
- $P_t = \mathbb{E}_t \left( \frac{1}{\beta_t} \int_t^{\bar{T}} \beta_s dD_s \right)$
- $\Delta_t = \int_{[t, \tau^{\delta}]} \beta_s dD_s$
- $\alpha_t = \exp ( - \int_0^t (r_s+\dot{\gamma}_s) ds )$

$\dot{\gamma}_s = \sum_{Y \in \mathcal{Y}_{\bullet}} \gamma_Y$ tous les chocs qui font tomber C ou B.

### Notations

Moreover, we used the following notations:

- $R_.$ the recovery of the bank or counterparty
- $\gamma^.$ the default intensity of the bank or counterparty

#### Collateral

The ***collateral*** of both parties are denoted by the following:

- $\Gamma^b$ the overall bank collateral posted by the counterparty: $\Gamma^b = VM + IM_c$, with $IM_c > 0$
- $-\Gamma^c$ the overall counterparty collateral posted by the bank: $\Gamma^c= VM + IM_b$, with $IM_b < 0$
- $-\Gamma^f = -\Gamma^c - IM_c$ the collateral funded by the bank (which is the point of view we adopted here). Note that the term $-IM_c$ corresponds to the rehypotecation of the initial margin posted (initially!) by the counterparty.

Note the the sign conventions:
$VM = +1$ means the counterparty owes $1$ to the bank. Moreover, $IM_.$ denotes the initial margin posted by $.$

#### Spread remunerations wrt to risk free rate

- $c_t$ the remuneration spread of the collateral
- $\lambda_t$ the investment spread of the bank
- $\tilde{\lambda}_t := \bar{\lambda}_t - (1-R_f)\gamma^b$ with
    - $\bar{\lambda}_t$ the unsecured funding spread of the bank
    - $R_f$ the recovery rate of the bank to its funder

### Comparison to CCVA

Compared to the CCVA computations, this notebook focuses on the most risky (member 0) trading with the counterparty 1.
Thus here:  

- the bank $\Leftrightarrow$ member n° 1
- the counterparty $\Leftrightarrow$ member n° 0

### Adding path

In [1]:
import sys

sys.path.append('../..')

## Monte Carlo `code`

### Simulation parameters

In [2]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.

# Moreover, we fix a $\delta$ mpor (margin period of risk) of 5 days (like in the CCP case)
mpor = 5*step

### Market parameters

In [3]:
from finance.discountfactor import ConstantRateDiscountFactor 

maturity = 10.
r = 0.02

discount = ConstantRateDiscountFactor(r)

### Exponential distribution (in order to compute $\mathbb{E} \int$)

In order to compute the integral term, we used the change of probability measure with the exponential density. In order to do so, we took $\lambda = \frac{2}{\bar{T}}$ in order to get the event $\{ \tau \leq \bar{T} \}$ with a probability $p = 1 - e^{-2} \approx 0.86$. Moreover, the distribution is centered to the middle of the interval $[0, \bar{T}]$.

In [4]:
from scipy.stats import expon

# To confirm, because 
lbda = 2.0 / maturity
exp_distrib = expon(loc=0, scale=1.0/lbda)

### Underlyings parameters

In [5]:
import numpy as np
from scipy.stats import norm
from maths.montecarlo.process.brownianmotion import BrownianMotion

time_grid = BrownianMotion.generate_time_grid(0, maturity, step)

x_0 = [100]
mu_s = [6]
sigma_s = [15]

b = BrownianMotion(x_0, mu_s, sigma_s, time_grid)

for i, (x0, m_, s_) in enumerate(zip(x_0, mu_s, sigma_s)):
    print "P(S_T^%d < 0) = %s"%(i+1, norm.cdf(-(x0 + m_ * maturity)/(s_*np.sqrt(maturity))))

P(S_T^1 < 0) = 0.000371639934425


### Swap Contract

In our simulations, we will use Swaps contracts with different maturities

In [6]:
from finance.products.european.swap import (
    SwapContract,
)

delta = 1.

swap_dates = SwapContract.generate_payment_dates(0, maturity, delta)
swap = SwapContract(b, discount, swap_dates)

price_0 = swap.price(0.)

print swap
print "\nPrice swap at t=0 = ",price_0

Swap contract of maturity T = 10 years, over S^0 with strike K = 109.237, paying at {0.00, 1.00, 2.00, 3.00, 4.00, 5.00, 6.00, 7.00, 8.00, 9.00, 10.00}

Price swap at t=0 =  2.27373675443e-13


### Exposures to derivatives for the $K_{b}$ computation

In [7]:
from risk.exposure import EuropeanQuantileBrownianExposure

exposure_mpor = 1./12
exposure_quantile = 0.99

index = swap.underlying_index
drift = mu_s[index]
vol = sigma_s[index]
exposure = EuropeanQuantileBrownianExposure(swap, exposure_mpor, drift, vol, exposure_quantile, discount)

### Marshall Olkin copula

We choose the following parameters:  

- $\gamma_0 = 500$ bps
- $\gamma_1 = 300$ bps
- $\gamma_2 = 100$ bps
- $\gamma_{0, 1} = 170$ bps
- $\gamma_{0, 1, 2} = 30$ bps

which leads to:

- $\gamma_0^{tot} = 700$ bps
- $\gamma_1^{tot} = 500$ bps
- $\gamma_2^{tot} = 130$ bps

Moreover, we choose all recoveries equal to $40\%$

In [8]:
from maths.copula.marshallolkin import MarshallOlkinCopula

nb_cm = 3

bank_index = 1
counterparty_index = 0
recoveries = [0.4 for i in range(nb_cm)]

mo_groups = [frozenset([i]) for i in range(nb_cm)]
mo_groups.append(frozenset([0, 1]))
mo_groups.append(frozenset([0, 1, 2]))

lambdas = [0.05, 0.03, 0.01, 0.017, 0.003]

copula = MarshallOlkinCopula(bank_index, nb_cm, mo_groups, lambdas, recoveries)

In [9]:
gamma_bank = copula.compute_gamma(bank_index)
print "Default intensity of the bank: %s"%gamma_bank

gamma_counterparty = copula.compute_gamma(counterparty_index)
print "Default intensity of the counterparty: %s"%gamma_counterparty

gamma_both = 0.
for group, lamb in zip(mo_groups, lambdas):
    if (bank_index in group) or (counterparty_index in group):
        gamma_both += lamb
        
print "\nDefault intensity of at least one of both: %s"%gamma_both

Default intensity of the bank: 0.05
Default intensity of the counterparty: 0.07

Default intensity of at least one of both: 0.1


In [10]:
alpha = ConstantRateDiscountFactor(r+gamma_both)

### Portfolio of the bank

#### Notional

In the paper of Crépey, Gerboud, Grbac and Ngor [Counterparty risk and funding:
the four wings of the TVA](http://grozny.maths.univ-evry.fr/pages_perso/crepey/papers/CVA-Wings.pdf), they compute the TVA on a swap with the following parameters (see section 4.4 Numerics):

>We choose a swap notional of $N = 310.136066$\$ so that the fixed leg of the swap is worth $100$\$ at inception

Let us compute the same way the notional such that the fixed leg is equal to $100$ \$.

\begin{equation*}
N = \frac{P_{fixed}}{K \times \sum_{k=1}^{N} \Delta_{T_k} \beta_{T_k}}
\end{equation*}

In [11]:
p_fixed = 100.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)
print "Notional =",notional

Notional = 0.102020134003


In [12]:
from finance.portfolio import Portfolio

members_quantities = np.array([[-.5],
                              [.5]])

members_positions = np.dot(members_quantities, notional).reshape(members_quantities.shape)
print members_positions

[[-0.05101007]
 [ 0.05101007]]


In [13]:
from finance.portfolio import Portfolio

portfolio = Portfolio(members_positions, [swap], [price_0], [exposure])

m_positions = np.matrix(members_positions)
portfolio_at_time_0 = m_positions*price_0
print portfolio_at_time_0

[[ -1.15983464e-14]
 [  1.15983464e-14]]


### Collateral management

#### Bank and counterparty's initial margins

In [14]:
from ccp.states import MembersState
from ccp.accounts import Accounts, DFAccounts

states = MembersState(2)
vm_accounts = Accounts(states)

In [15]:
conf_level_im = 0.90
min_im_required = 0.
im_accounts = Accounts(states)
im_accounts.reset(min_im_required)

### Default model of the counterparty

In [16]:
from credit.default_probability import FlatIntensity

default_model = FlatIntensity(gamma_counterparty)

### Regulatory capital

In [17]:
from risk.basel import RegulatoryCapital

#From Kenyon [KVA] p.13: c = 8%

kenyon_c = .08

basel_capital = RegulatoryCapital(vm_accounts, 
                                  im_accounts, 
                                  portfolio,
                                  recovery=recoveries[counterparty_index], 
                                  default_proba=default_model,
                                  c=kenyon_c)

### Simulations

In [18]:
intensities = {"b": gamma_bank, "c": gamma_counterparty}
recov = {"b": recoveries[bank_index], "c": recoveries[counterparty_index], "f": 1.0}

#### Definition of the function $\tilde{f}_{.}(0)$

We consider here $(c_t), (\lambda_t), (\bar{\lambda}_t)$ and $(k_t)$ as constant processes.

In [19]:
c_ = 0.0150
lambda_ = 0.0150
bar_lambda_ = 0.0450
k_ = 0.1

def tilde_f(p, capital, ims, recoveries, intensities):    
    #gamma_c = p+ims["b"]
    #gamma_b = p+ims["c"]
    
    gamma_c = +ims["b"]
    gamma_b = +ims["c"]
    
    cva = (1.-recoveries["c"]) * intensities["c"] * np.maximum(p-gamma_c, 0)
    dva = +(1.-recoveries["b"]) * intensities["b"] * np.minimum(p-gamma_b, 0)
    
    gamma_f = gamma_c+ims["c"]
    tilde_lambda = bar_lambda_ - (1.-recoveries["f"])*intensities["b"]
    
    fva = c_*gamma_f + tilde_lambda*np.maximum(p-gamma_f, 0) + lambda_*np.minimum(p-gamma_f, 0)
    
    kva = k_*capital
    
    return cva, dva, fva, kva

In [20]:
N = 10000

#### Monte Carlo loop

In [21]:
import time

start_time = time.time()

# CVA part
cva = 0.
cva_square = 0.   

# DVA part
dva = 0.
dva_square = 0.   

# FVA part
fva = 0.
fva_square = 0.    

# KVA part
kva = 0.
kva_square = 0.

random_times = exp_distrib.rvs(size=N)

for tau in random_times:
    if tau > maturity:
        continue
    
    # Date of the time_grid right after tau
    idx = np.searchsorted(time_grid, tau, side='right')
    t = time_grid[idx]

    # Last pillar of the swap (need to compute it to have the quantile price)
    last_pill_swap_idx = np.searchsorted(swap.pillars, t) - 1
    last_pill_swap = swap.pillars[last_pill_swap_idx]
    
    # We generate the brownian path
    tmp_time = [0, last_pill_swap, t]
    b = BrownianMotion(x_0, mu_s, sigma_s, tmp_time)
    
    # We compute now the swap price with the new underlying ...
    swap.set_underlying(b)
    swap_price = swap.price(t)

    p_and_l = portfolio.compute_pl([swap_price])
    
    # For the regulatory capital
    #for ii, pl in enumerate(p_and_l):
    #    vm_accounts.put_amount(ii, pl)
    
    p = p_and_l[bank_index]
    
    # ... and then the collateral
    #collats = portfolio.compute_exposure(t, risk_period=mpor, conf_level=conf_level_im)
    #for ii, gamma in enumerate(collats.flat):
    #    #tmp = np.maximum(gamma-swap_price, min_im_required)
    #    tmp = gamma
    #    im_accounts.put_amount(ii, tmp)
    
    # Computation of the RWA
    rwa = basel_capital.compute_ccr(bank_index, t)
    
    #ims = { "b": -im_accounts.get_amount(bank_index), # J
    #        "c": im_accounts.get_amount(counterparty_index)} # I
    ims = {"b": 0, "c": 0}
    
    # Computation of the different terms
    __cva, __dva, __fva, __kva = tilde_f(p, rwa, ims, recov, intensities)
    
    # Computation of the alpha times inv density
    inv_pdf = 1./exp_distrib.pdf(t)
    alpha_tau = alpha(t)    
    tmp_factor = alpha_tau * inv_pdf
        
    _cva = __cva*tmp_factor
    _dva = __dva*tmp_factor
    _kva = __kva*tmp_factor
    _fva = __fva*tmp_factor
    
    cva += _cva
    cva_square += _cva**2
    
    dva += _dva
    dva_square += _dva**2
            
    fva += _fva
    fva_square += _fva**2
            
    kva += _kva
    kva_square += _kva**2  

exec_time = time.time() - start_time

In [22]:
# 1st index: CVA
# 2nd index: DVA
# 3st index: FVA
# 4th index: KVA

tot_cva = cva
tot_dva = dva
tot_fva = fva
tot_kva = kva

tot_cva2 = cva_square
tot_dva2 = dva_square
tot_fva2 = fva_square
tot_kva2 = kva_square

labels = ['CVA', 'DVA', 'FVA', 'KVA']
expectations = []
mod_variances = []

e_cva = tot_cva / N
e_dva = tot_dva / N
e_fva = tot_fva / N
e_kva = tot_kva / N

expectations.append(e_cva)
expectations.append(e_dva)
expectations.append(e_fva)
expectations.append(e_kva)

mod_v_cva = (tot_cva2/N - e_cva**2) / (N-1)
mod_v_dva = (tot_dva2/N - e_dva**2) / (N-1)
mod_v_fva = (tot_fva2/N - e_fva**2) / (N-1)
mod_v_kva = (tot_kva2/N - e_kva**2) / (N-1)

mod_variances.append(mod_v_cva)
mod_variances.append(mod_v_dva)
mod_variances.append(mod_v_fva)
mod_variances.append(mod_v_kva)

In [25]:
# Confidence Interval results:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Bilateral case comparison"
for i in range(30):
    print "-",

print "\n\nResults for %d iterations"%(N)
print "Execution time: %s sec"%exec_time

print "\nUsed discount factor: %s"%discount

print "\nDefault intensity of the bank = %d bps"%(gamma_bank*1e4)
print "Default intensity of the counterparty = %d bps\n"%(gamma_counterparty*1e4)

print swap

print "\nPrice of the swap at t=0: %s"%price_0
print "Price of the fixed leg at t=0: %s"%(np.dot(swap.delta_time, [discount(t) for t in swap.pillars[1:]]) * swap.strike * notional)
print "\nQuantities bought:\n %s\n"%members_positions
print "Portfolio values of actors contracted:\n %s \n"%portfolio_at_time_0

#print "\nConfidence level used for IM: %.2f\n"%conf_level_im

for label, e, mv in zip(labels, expectations, mod_variances):
    half_inter = z_level*np.sqrt(mv)
    print "The %s of the bank lies in the interval: [%.4f, %.4f]"%(label, e-half_inter, e+half_inter)

Bilateral case comparison
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Results for 10000 iterations
Execution time: 18.754999876 sec

Used discount factor: Constant discount factor process with rate r = 0.02

Default intensity of the bank = 500 bps
Default intensity of the counterparty = 700 bps

Swap contract of maturity T = 10 years, over S^0 with strike K = 109.237, paying at {0.00, 1.00, 2.00, 3.00, 4.00, 5.00, 6.00, 7.00, 8.00, 9.00, 10.00}

Price of the swap at t=0: 2.27373675443e-13
Price of the fixed leg at t=0: 100.0

Quantities bought:
 [[-0.05101007]
 [ 0.05101007]]

Portfolio values of actors contracted:
 [[ -1.15983464e-14]
 [  1.15983464e-14]] 

The CVA of the bank lies in the interval: [1.3411, 1.4123]
The DVA of the bank lies in the interval: [-0.2345, -0.2126]
The FVA of the bank lies in the interval: [1.3231, 1.4034]
The KVA of the bank lies in the interval: [0.3344, 0.3418]
